## 📘 How to Use Kaggle (Upload Dataset & Notebook)

### ✅ Step 1: Create Kaggle Account
- Go to 👉 https://www.kaggle.com  
- Sign in using Google / Email

---

### ✅ Step 2: Upload Your Dataset
1. Click **Datasets** → **Create New Dataset**
2. Upload your **dataset folder or ZIP file**
3. Add:
   - Dataset name
   - Short description
4. Set visibility → **Public / Private**
5. Click **Create**

✅ After upload, Kaggle gives a dataset path like:


## 🟢 Cell 1 – Imports, Paths & Data Generators

This cell:

- Imports core libraries:
  - **TensorFlow / Keras** → modeling & training
  - **ImageDataGenerator** → image loading + augmentation
  - **Sklearn metrics** → evaluation (report + confusion matrix)
  - **Matplotlib / Seaborn** → visualization

- Defines dataset paths:
  - `train_dir`, `val_dir`, `test_dir` inside the **Diabetic Retinopathy** dataset folder.

- Sets image & training parameters:
  - `IMG_HEIGHT = 224`, `IMG_WIDTH = 224`, `BATCH_SIZE = 32`
  - `NUM_CLASSES = 2` → DR vs No_DR (binary)

- Creates **data generators**:
  - `train_datagen` with:
    - rescaling (0–1)
    - rotation, shift, zoom, horizontal flip → robust to variations
  - `val_datagen`, `test_datagen` with just rescaling

- Builds **directory-based generators**:
  - `train_generator`, `val_generator`, `test_generator`
  - `class_mode='binary'` → labels are 0 or 1
  - `shuffle=False` for test generator → keeps order for evaluation

- Prints `train_generator.class_indices` to show class → label mapping.

🔗 Official docs (used once for entire code):

- Keras ImageDataGenerator: https://keras.io/api/preprocessing/image/  
- Model training API: https://keras.io/api/models/model_training_apis/  
- Sklearn metrics: https://scikit-learn.org/stable/modules/model_evaluation.html  


#### Dataset path: https://www.kaggle.com/datasets/huebitsvizg/retinopathy-detection

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import itertools

# Paths — adjust as per your folder structure
base_dir = '/kaggle/input/diagnosis-of-diabetic-retinopathy/Diagnosis of Diabetic Retinopathy'   # change this to your root folder
train_dir = os.path.join(base_dir, 'train')
val_dir   = os.path.join(base_dir, 'valid')
test_dir  = os.path.join(base_dir, 'test')

# Image parameters
IMG_HEIGHT = 224
IMG_WIDTH  = 224
BATCH_SIZE = 32
NUM_CLASSES = 2    # DR vs No_DR

# Data generators with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen= ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'     # since two classes
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False    # for evaluation, keep ordering
)

print("Classes:", train_generator.class_indices)


## 🟢 Cell 2 – CNN Model Definition & Compilation

This cell defines a **simple CNN** for binary DR classification:

- Architecture:
  - `Input(shape=(224, 224, 3))`
  - Conv2D(32) → MaxPooling
  - Conv2D(64) → MaxPooling
  - Conv2D(128) → MaxPooling  
  → gradual increase in filters = more complex pattern learning
  - `Flatten()` → converts feature maps to a vector
  - `Dense(128, relu)` → learns high-level features
  - `Dropout(0.5)` → reduces overfitting
  - `Dense(1, sigmoid)` → outputs a probability for "DR present"

- Compilation:
  - Optimizer: `adam`
  - Loss: `binary_crossentropy` (since binary classification)
  - Metric: `accuracy`

- `model_cnn.summary()` prints:
  - Layer types
  - Output shapes
  - Number of parameters

✅ This creates and prepares the CNN for training.


In [ ]:
# Define the model
model_cnn = keras.Sequential([
    layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')   # binary classification
])

model_cnn.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_cnn.summary()


### Training the model

In [ ]:
epochs = 10   # you can increase
history_cnn = model_cnn.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    epochs=epochs,
    validation_data = val_generator,
    validation_steps = val_generator.samples // BATCH_SIZE
)


## 🟢 Cell 3 – Prediction, Classification Report & Confusion Matrix

This cell **evaluates** the CNN:

- `model_cnn.predict(test_generator, steps=test_steps)`:
  - Gets predicted probabilities for each test image.
- Converts probabilities to class labels:
  - `pred_labels = (pred_probs > 0.5).astype(int)`  
  → threshold at 0.5:  
    - ≥ 0.5 → class 1 (DR)  
    - < 0.5 → class 0 (No_DR)

- `true_labels = test_generator.classes`:
  - True labels in same order (because `shuffle=False`).

- `classification_report(...)`:
  - Shows precision, recall, F1-score and support per class.

- `confusion_matrix(true_labels, pred_labels)`:
  - Counts TP, TN, FP, FN
  - Plotted using Seaborn heatmap:
    - X-axis: Predicted
    - Y-axis: Actual

✅ This gives a clear picture of **how well the CNN distinguishes DR vs No_DR**, beyond just accuracy.


In [ ]:
# Get predictions
test_steps = test_generator.samples // BATCH_SIZE + 1
pred_probs = model_cnn.predict(test_generator, steps=test_steps)
pred_labels = (pred_probs > 0.5).astype(int).reshape(-1)

true_labels = test_generator.classes  # since shuffle=False
class_names = list(test_generator.class_indices.keys())

# Print classification report
print(classification_report(true_labels, pred_labels, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

sns.heatmap(cm, annot=True, fmt='d', cmap='Purples')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("CNN Confusion Matrix")
plt.show()


In [ ]:
from tensorflow.keras.models import load_model

# Save as HDF5
model_cnn.save('cnn_model.h5')
print("CNN model saved as cnn_model.h5")

# Load later
loaded_cnn_h5 = load_model('cnn_model.h5')

# Run this in colab

##  Environment Setup for Model Deployment (Flask + Ngrok)

This cell prepares the system for **web deployment of your deep learning model**.

###  Installed Packages
- **Flask** → Web framework for building the prediction UI  
  https://flask.palletsprojects.com/
- **pyngrok** → Creates public URL for Colab/Notebook apps  
  https://ngrok.com/docs
- **TensorFlow** → Loads and runs your trained CNN model  
  https://www.tensorflow.org/
- **Pillow (PIL)** → Handles image loading and processing  
  https://python-pillow.org/

###  Folder Creation
Creates required directories:
- `templates/` → HTML files
- `static/` → CSS, JS files
- `uploads/` → Uploaded images

 This cell ensures your system is **ready for model-based web deployment**.


In [ ]:
!pip install flask pyngrok tensorflow pillow
!mkdir -p templates static uploads

**Connects drive to colab for easy access to the files**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Verify model file path
!ls "/content/drive/MyDrive/Colab Notebooks/Varma sir projects/cnn_advanced_model.h5"

##  Diabetic Retinopathy Prediction – Flask Backend (API)

This Flask app deploys your **trained CNN diabetic retinopathy model** and provides a web-based prediction system.

###  What This App Does
- Loads the trained **CNN model (`.h5`)** from Google Drive  
- Accepts eye images uploaded by the user  
- Preprocesses images to **224×224 + normalization**  
- Predicts:
  - **DR** → Diabetic Retinopathy detected  
  - **No_DR** → Healthy retina  
- Returns a **user-friendly message with medical guidance**

---

###  Key Routes
- `/` → Welcome page  
- `/upload` → Image upload page  
- `/predict` → Model inference (POST request)

---

###  Prediction Logic
- Image → Resize → Normalize → CNN Prediction  
- Output > 0.5 → **No_DR**  
- Output ≤ 0.5 → **DR**

---

###  Libraries Used
- **Flask** – Web framework → https://flask.palletsprojects.com/  
- **TensorFlow/Keras** – Model loading & prediction → https://www.tensorflow.org/  
- **Pillow (PIL)** – Image handling → https://python-pillow.org/

This file acts as the **core backend engine** for your diabetic retinopathy web application.


In [ ]:
%%writefile app.py
import os
import tensorflow as tf
from flask import Flask, request, render_template, jsonify, redirect, url_for
from PIL import Image
import numpy as np

# Load the saved CNN diabetic retinopathy model
model_path = "/content/drive/MyDrive/Colab Notebooks/Varma sir projects/cnn_advanced_model.h5"
model = tf.keras.models.load_model(model_path)

app = Flask(__name__)

def preprocess_image(image):
    image = image.resize((224, 224))  # Update from (128,128) to correct size
    image = np.array(image) / 255.0
    return np.expand_dims(image, axis=0)


@app.route("/")
def welcome():
    return render_template("welcome.html")

@app.route("/upload")
def upload_page():
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def predict():
    if "image" not in request.files:
        return jsonify({"error": "No image uploaded"}), 400
    file = request.files["image"]
    try:
        image = Image.open(file).convert("RGB")
        input_arr = preprocess_image(image)
        pred = model.predict(input_arr)[0][0]
        # Adjust label logic consistently with training
        label = "No_DR" if pred > 0.5 else "DR"

        # Friendly explanations with emojis
        explanations = {
            "No_DR": "👁️ Your retina looks healthy with no signs of diabetic retinopathy detected. Keep up good eye care habits!",
            "DR": "⚠️ Signs of diabetic retinopathy detected. It's advisable to consult an eye specialist early to prevent vision loss."
        }
        explanation_text = explanations.get(label, "Prediction unavailable.")
        return jsonify({"prediction": explanation_text})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=True)


##  Retina Image Upload & Prediction UI (Frontend)

This page provides the **user interface** for uploading a retina image and displaying the CNN prediction result.

###  What This Page Does
- Allows users to **upload a retina eye image**
- Shows a **live image preview** before prediction
- Sends the image to Flask using **AJAX (Fetch API)**
- Displays the **AI prediction result dynamically (without page reload)**

---

###  Key Components
- **HTML Form** → Uploads retina image
- **JavaScript (Fetch API)** → Sends image to `/predict`
- **Live Preview** → Displays selected image instantly
- **Result Box** → Shows prediction returned by the CNN model

---

###  Technologies Used
- **HTML5** – Page structure → https://developer.mozilla.org/en-US/docs/Web/HTML  
- **JavaScript Fetch API** – Image upload & prediction → https://developer.mozilla.org/en-US/docs/Web/API/Fetch_API  
- **Flask Jinja (`url_for`)** – Static file linking → https://flask.palletsprojects.com/

 This file acts as the **frontend bridge between user and deep learning model**.


In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>👁️ Diabetic Retinopathy Prediction</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}" />
</head>
<body>
  <div class="container">
    <header class="header">
      <h1>👁️ Upload Retina Image for Detection</h1>
      <p class="subtitle">Select an eye retina image, then click 'Detect' to see the results.</p>
    </header>
    <div class="card">
      <form id="uploadForm" onsubmit="uploadImage(event)">
        <label for="image">📸 Choose Retina Image:</label>
        <input type="file" id="image" name="image" accept="image/*" required />
        <button type="submit" class="submit-btn">🔍 Detect</button>
      </form>
      <img id="uploadedImage" alt="Preview" style="display:none;"/>
      <div id="predictionResult" class="result-text"></div>
    </div>
  </div>

<script>
function uploadImage(event) {
  event.preventDefault();
  const file = document.getElementById("image").files[0];
  if (!file) {
    alert("Please select an image file.");
    return;
  }

  const reader = new FileReader();
  reader.onload = e => {
    const preview = document.getElementById("uploadedImage");
    preview.src = e.target.result;
    preview.style.display = "block";
  };
  reader.readAsDataURL(file);

  const formData = new FormData();
  formData.append("image", file);
  document.getElementById("predictionResult").textContent = "🔎 Detecting...";

  fetch("/predict", {
    method: "POST",
    body: formData
  }).then(res => res.json())
    .then(data => {
      if(data.error){
        document.getElementById("predictionResult").textContent = "❌ " + data.error;
      } else {
        document.getElementById("predictionResult").innerHTML = "🧠 <strong>Prediction:</strong> " + data.prediction;
      }
    }).catch(() => {
      document.getElementById("predictionResult").textContent = "⚠️ Error during prediction.";
    });
}
</script>
</body>
</html>


##  Welcome Page – Diabetic Retinopathy Detection App

This page serves as the **landing screen** of the application.

###  Purpose
- Introduces the **DR Detection System**
- Briefly explains the **importance of early eye disease detection**
- Provides a **navigation button** to the image upload page (`/upload`)

###  Key Features
- Simple **call-to-action button**
- Clean **user-friendly introduction**
- Styled using external **CSS via Flask (`url_for`)**

###  Technologies Used
- **HTML5** → Page structure  
  https://developer.mozilla.org/en-US/docs/Web/HTML  
- **Flask Jinja** → Static file linking  
  https://flask.palletsprojects.com/

 This page acts as the **entry point for users before prediction**.


In [ ]:
%%writefile templates/welcome.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>👋 Welcome to DR Detection</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}" />
</head>
<body>
  <div class="welcome-container">
    <h1>👁️ Welcome to Diabetic Retinopathy Detection App</h1>
    <p class="subtitle">Early detection helps protect your vision. Upload a retina image to get started!</p>
    <button onclick="location.href='/upload'" class="welcome-btn">➡️ Go to Upload Page</button>
  </div>
</body>
</html>


## 🎨 UI Styling – Diabetic Retinopathy Web App (CSS)

This file designs the **entire visual theme** of your retina detection web application.

###  What This CSS Controls
- **Dark blue medical gradient theme** for a professional healthcare look  
- **Upload card styling** with hover animations  
- **Prediction result highlight box**  
- **Image preview styling**  
- **Welcome page layout & button animations**

###  Key Design Features
- Gradient backgrounds for **medical UI feel**
- Animated transitions using:
  - `fadeIn`
  - `fadeInDown`
- Responsive containers for **mobile & desktop**
- Styled buttons for:
  - Upload (`Detect`)
  - Navigation (`Welcome` button)

###  Technology Used
- **Pure CSS3 Animations & Layouts**  
  https://developer.mozilla.org/en-US/docs/Web/CSS  

 This CSS file ensures your **AI medical application looks clean, responsive, and professional**.


In [ ]:
%%writefile static/style.css
body {
  font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
  background: linear-gradient(135deg, #001f4d 0%, #003366 100%); /* Navy blue gradient */
  color: #e6eaf0;
  padding: 20px;
  min-height: 100vh;
  margin: 0;
}

/* Container for upload page */
.container {
  max-width: 700px;
  margin: 0 auto;
  padding: 20px;
}

.header {
  text-align: center;
  margin-bottom: 30px;
  animation: fadeInDown 1s ease forwards;
}

.header h1 {
  font-size: 3rem;
  margin-bottom: 10px;
  letter-spacing: 1px;
  text-shadow: 1px 1px 5px rgba(0,0,0,0.7);
}

.header p {
  font-size: 1.3rem;
  color: #a0b8d7;
}

.card {
  background: #f0f4ff;
  color: #222;
  border-radius: 20px;
  padding: 40px;
  box-shadow: 0 15px 40px rgba(0,0,0,0.3);
  animation: fadeIn 1s ease forwards;
  margin-bottom: 25px;
  transition: transform 0.3s ease;
}

.card:hover {
  transform: translateY(-6px);
  box-shadow: 0 20px 50px rgba(0,0,0,0.4);
}

.result-text {
  background: #dde7f9;
  padding: 20px;
  border-radius: 15px;
  border-left: 8px solid #3a5a99;
  margin-top: 20px;
  font-size: 1.4rem;
  color: #1a2d59;
  font-weight: 600;
  text-align: center;
  box-shadow: inset 0 0 10px rgba(0,0,0,0.08);
  transition: background 0.3s, box-shadow 0.3s;
}

input[type='file'] {
  width: 100%;
  padding: 16px;
  border: 3px dashed #3a5a99;
  border-radius: 15px;
  margin-bottom: 20px;
  font-size: 1rem;
  cursor: pointer;
  background: #f7faff;
  transition: border-color 0.3s;
}

input[type='file']:hover {
  border-color: #2c4579;
}

.submit-btn, .welcome-btn {
  width: 100%;
  padding: 20px;
  background: linear-gradient(135deg, #004080, #336699);
  border: none;
  color: #f0f4ff;
  font-size: 1.2rem;
  font-weight: 700;
  border-radius: 15px;
  cursor: pointer;
  box-shadow: 0 8px 15px rgba(0, 64, 128, 0.7);
  transition: transform 0.3s, box-shadow 0.3s;
  margin-top: 10px;
}

.submit-btn:hover, .welcome-btn:hover {
  transform: translateY(-4px);
  box-shadow: 0 12px 25px rgba(0, 64, 128, 1);
}

#uploadedImage {
  display: block;
  max-width: 100%;
  margin-top: 20px;
  border-radius: 15px;
  border: 4px solid #3a5a99;
  box-shadow: 0 4px 8px rgba(0,0,0,0.2);
  animation: fadeIn 1s ease forwards;
}

.welcome-container {
  max-width: 600px;
  margin: 5% auto;
  background: rgba(10, 30, 70, 0.9);
  padding: 50px;
  border-radius: 25px;
  text-align: center;
  box-shadow: 0 20px 50px rgba(0,0,0,0.8);
  animation: fadeIn 1s ease forwards;
}

.welcome-container h1 {
  font-size: 4rem;
  margin-bottom: 20px;
  color: #c1d1f5;
  text-shadow: 2px 2px 12px rgba(0,0,0,0.6);
}

.welcome-container .subtitle {
  font-size: 1.6rem;
  margin-bottom: 30px;
  color: #a8bae0;
  line-height: 1.4;
}

/* Animations */
@keyframes fadeIn {
  from {opacity: 0; transform: translateY(20px);}
  to {opacity:1; transform: translateY(0);}
}

@keyframes fadeInDown {
  from {opacity:0; transform: translateY(-30px);}
  to {opacity:1; transform: translateY(0);}
}

@keyframes bounceIn {
  0%, 20%, 40%, 60%, 80%, 100% {
    animation-timing-function: cubic-bezier(0.215, 0.610, 0.355, 1.000);
  }
  0% {opacity: 0; transform: scale3d(.3, .3, .3);}
  20% {transform: scale3d(1.1, 1.1, 1.1);}
  40% {transform: scale3d(.9, .9, .9);}
  60% {opacity: 1; transform: scale3d(1.03, 1.03, 1.03);}
  80% {transform: scale3d(.97, .97, .97);}
  100% {opacity: 1; transform: scale3d(1, 1, 1);}
}


##  Stop Running Flask & Ngrok Processes (Port Cleanup)

This cell **safely terminates any previously running Flask or Ngrok services** to avoid:
-  Port conflicts  
-  Duplicate servers  
-  Background process errors  

### 🔹 What Each Command Does
- `pkill -f flask` → Stops any active Flask server  
- `pkill -f ngrok` → Stops any running Ngrok tunnel  
- `|| echo "No ... running"` → Prevents errors if nothing is running  

In [ ]:
# Kill any previous processes
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

## 🔍 Check Which Process Is Using Port 8000

This command identifies the **running process currently using port 8000**.

### 🔹 What Each Part Means
- `lsof` → Lists open files & network connections  
- `-i` → Filters network processes  
- `:8000` → Checks only **port 8000**  

✅ Use this to:
- Confirm whether your **Flask server is running**
- Find the **Process ID (PID)** before stopping it


In [ ]:
!lsof -i :8000

In [ ]:
!kill -9 7187 7242


##  Run Flask App in Background (Production Mode)

This command starts your **Flask web server in the background** so it keeps running even if the notebook cell finishes.

### 🔹 What Each Part Does
- `nohup` → Keeps the server running after the session continues  
- `python app.py` → Launches your Flask application  
- `> flask.log` → Saves all server logs to a file  
- `2>&1` → Captures both errors and outputs in the log  
- `&` → Runs the process in the background  

In [ ]:
# Run Flask in the background
!nohup python app.py > flask.log 2>&1 &


 Ngrok Setup

Ngrok provides a public HTTPS link.

Your ngrok token was removed for safety.

To use ngrok:
1. Get token → https://dashboard.ngrok.com/get-started/your-authtoken  
2. Add inside notebook:

conf.get_default().auth_token = "YOUR_NGROK_TOKEN_HERE"

3. Start tunnel:

public_url = ngrok.connect(8000)

Shareable app link appears here.


In [ ]:
# Start ngrok tunnel
from pyngrok import ngrok, conf
conf.get_default().auth_token = "place your ngrok token here"

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

# Optional: check logs
!sleep 3 && tail -n 20 flask.log


## 📄 View Latest Flask Server Logs

This command displays the **last 50 lines** of your Flask server log file.

### 🔹 What It Does
- `tail` → Reads the last part of a file  
- `-n 50` → Shows the **most recent 50 log entries**  
- `flask.log` → Log file where Flask outputs are saved  

In [ ]:
!tail -n 50 flask.log